In [14]:
import numpy as np
import pandas as pd
import yfinance as yf

In [15]:
# SET PARAMETERS
ticker_symbol = 'GOOG' # can also be 'AAPL GOOG AMZN'
start_date = '2021-01-01'
end_date = '2023-12-30'

df = yf.download(ticker_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [16]:
def get_macd(df, is_daily, is_stock):
    if is_daily:
        factor = 1
        col_prefix = 'daily'
    else:
        factor = 5 if is_stock else 7
        col_prefix = 'weekly'
    # Obtain the moving averages for the MACD
    slow_ma = df['Close'].ewm(span=factor*26, adjust=False).mean()
    fast_ma = df['Close'].ewm(span=factor*12, adjust=False).mean()
    # Calculate the MACD and then use that to generate the signal line
    
    df.loc[:, col_prefix + '_macd'] = fast_ma - slow_ma
    df.loc[:, col_prefix + '_signal'] = (
        (fast_ma - slow_ma)
        .ewm(span=factor*9, adjust = False)
        .mean()
    )

    return df.dropna()

In [17]:
df = get_macd(df, is_daily=True, is_stock=True)
df = get_macd(df, is_daily=False, is_stock=True)

In [18]:
df.to_csv('output.csv')

In [19]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,daily_macd,daily_signal,weekly_macd,weekly_signal
Date,,,,,,,,,,
2021-01-04,87.876999,88.032501,85.392502,86.412003,86.412003,38038000,0.000000,0.000000,0.000000,0.000000
2021-01-05,86.250000,87.383499,85.900749,87.045998,87.045998,22906000,0.050575,0.010115,0.011107,0.000483
2021-01-06,85.131500,87.400002,84.949997,86.764503,86.764503,52042000,0.067168,0.021526,0.016749,0.001190
2021-01-07,87.002998,89.419998,86.852501,89.362503,89.362503,45300000,0.286650,0.074550,0.067631,0.004079
2021-01-08,89.399002,90.491997,88.676750,90.360497,90.360497,41012000,0.534954,0.166631,0.133543,0.009708


In [33]:
def backtest(Open, Close, Low, weekly_opp, daily_opp):
    '''
    Perform the buying/selling. The strategy is as follows:
    1. Buy if the weekly MACD > weekly signal and daily MACD > daily signal
    2. Wait until the daily signal > daily MACD, set the SL to the LOD
    3. Update the stop loss if the MACD crosses below the signal again
    Parameters
    Open : np.array
    The open prices over each day
    Close: np.array,
    The close prices for each day
    Low : np.array
    The low prices over each day weekly_opp: np.array
    An array to show when the weekly MACD is above the weekly signal daily_opp: np.array
    An array to show when the daily MACD is above the daily signal
    Returns
    bought, sold, percs: np.array
    The outcome from the backtest
    '''
    # Empty list to store the simulation outcome
    percs = []
    # Flags for the backtest. Holding is used to flag whether we are currently
    # holding an asset. The sl_set is used to determine if we the
    # stop loss has been fixed and we waiting for it to be hit.
    holding = False
    sl_set = False
    for idx in range(1, Open.shape[0]):
        print(holding)
        # Check the previous day, and see if a buy signal was met. If so, enter on the open
        # and we're in the trade for at least the first day
        if weekly_opp[idx-1] and daily_opp[idx-1] and not holding:
            percs. append(Close[idx]/Open[idx]-1)
            holding = True
            continue
        elif holding:
            # In this case, the daily MACD < daily signal, and we set the SL
            # to the day's low price
            if not sl_set and not daily_opp[idx]:
                sl_set = True
                sl_val = Low[idx]
                percs. append(Close[idx]/Close[idx-1]-1)
                continue
            # If the SL was not hit, and the MACD crosses again, we update the SL
            elif (
            sl_set
            and Low[idx] > sl_val 
            and daily_opp[idx-1]
            and not daily_opp[idx]
            ):
                sl_val = Low[idx]
                percs.append(Close[idx]/Close[idx-1]-1)
                continue
            # In this case, the stop loss is met, and we exit the trade
            elif sl_set and Low[idx] < sl_val:
                holding = False
                sl_set = False
                percs.append(sl_val/Close[idx-1]-1)
                del sl_val
                continue
            else:
                percs. append(Close[idx]/Close[idx-1]-1)
        else:
            percs. append (0)
    return np.array(percs)

In [34]:
# df.loc[1:, 'percs'] = backtest(
#     df['Open'].values,
#     df['Close'].values,
#     df['Low'].values,
# (df['weekly_macd'] > df['weekly_signal']).values,
# (df['daily_macd'] > df['daily_signal']). values,
# )

percs = backtest(
    df['Open'].values,
    df['Close'].values,
    df['Low'].values,
    (df['weekly_macd'] > df['weekly_signal']).values,
    (df['daily_macd'] > df['daily_signal']).values
)

# Assign the values to the 'percs' column starting from row 2
df['percs'] = np.nan  # Initialize the column with NaN
df['percs'][1:] = percs  # Assign values starting from row 2

df['equity_strat'] = np.cumprod(1+df['percs'])
df['equity_bh'] = np. cumprod(1+df['Close'].pct_change())

False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False


/var/folders/bc/440d68zj0qgfv8fdlqk34yyr0000gn/T/ipykernel_72631/3355223222.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['percs'][1:] = percs  # Assign values starting from row 2


In [35]:
df.head(20)

,Open,High,Low,Close,Adj Close,Volume,daily_macd,daily_signal,weekly_macd,weekly_signal,percs,equity_strat,equity_bh
Date,,,,,,,,,,,,,
2021-01-04,87.876999,88.032501,85.392502,86.412003,86.412003,38038000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2021-01-05,86.250000,87.383499,85.900749,87.045998,87.045998,22906000,0.050575,0.010115,0.011107,0.000483,0.000000,1.000000,1.007337
2021-01-06,85.131500,87.400002,84.949997,86.764503,86.764503,52042000,0.067168,0.021526,0.016749,0.001190,0.019182,1.019182,1.004079
2021-01-07,87.002998,89.419998,86.852501,89.362503,89.362503,45300000,0.286650,0.074550,0.067631,0.004079,0.029943,1.049700,1.034145
2021-01-08,89.399002,90.491997,88.676750,90.360497,90.360497,41012000,0.534954,0.166631,0.133543,0.009708,0.011168,1.061423,1.045694
2021-01-11,89.303497,89.715752,88.026001,88.335999,88.335999,24194000,0.561900,0.245685,0.160786,0.016276,-0.022405,1.037642,1.022265
2021-01-12,87.695999,88.902000,86.265503,87.327499,87.327499,27140000,0.496158,0.295780,0.168984,0.022916,-0.011417,1.025795,1.010595
2021-01-13,86.929001,88.251747,86.900497,87.720001,87.720001,21882000,0.470307,0.330685,0.183585,0.029901,0.004495,1.030406,1.015137
2021-01-14,87.681000,88.750504,86.669998,87.009003,87.009003,23590000,0.387976,0.342143,0.184942,0.036642,-0.008105,1.022054,1.006909
